## Training pretrained model on coco with Pascal dataset

Importing mmseg and other necessary packages

In [10]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.6.0 True
0.18.0


In [17]:
torch.cuda.empty_cache()

In [18]:
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
from mmcv import Config

In [19]:
# File already downloaded into checkpoints folder

# !wget https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3/deeplabv3_r101-d8_512x512_4x4_20k_coco-stuff10k/deeplabv3_r101-d8_512x512_4x4_20k_coco-stuff10k_20210821_043025-c49752cb.pth -P checkpoints/

# !wget https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3/deeplabv3_r50-d8_512x512_4x4_20k_coco-stuff10k/deeplabv3_r50-d8_512x512_4x4_20k_coco-stuff10k_20210821_043025-b35f789d.pth -P checkpoints/

In [28]:
#using pretrained model on coco dataset
config_file = 'configs/deeplabv3/deeplabv3_r50-d8_512x512_4x4_20k_coco-stuff10k.py'
checkpoint_file = 'checkpoints/deeplabv3_r50-d8_512x512_4x4_20k_coco-stuff10k_20210821_043025-b35f789d.pth'

## Transfer learning

Using the model trained on Coco dataset to train on Pascal dataset by transfer learning and test the performance. 

Since the given config is used to train deeplabv3 on cocostuff dataset, we need to modify it accordingly for our ade dataset.  

In [29]:
# Config is loaded
cfg = Config.fromfile(config_file)

In [22]:
# Modifying the config file for our dataset:
from mmseg.apis import set_random_seed


# Since we use ony one GPU, BN is used instead of SyncBN
cfg.norm_cfg_1 = dict(type='BN', requires_grad=False)
cfg.norm_cfg_2 = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg_1        # Freezing lower layers
cfg.model.decode_head.norm_cfg = cfg.norm_cfg_1 
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg_2

# modify num classes of the model in decode/auxiliary head - classes - 150 in ade
cfg.model.decode_head.num_classes = 150 
cfg.model.auxiliary_head.num_classes = 150 

# Modify dataset type and path
cfg.dataset_type = 'ADE20KDataset'
cfg.data_root = 'data/ade/ADEChallengeData2016'

# cfg.data.samples_per_gpu = 4
# cfg.data.workers_per_gpu= 4

cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (512, 512)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(2048, 1024), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(2048, 512),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


cfg.data = dict(
    samples_per_gpu= 4,
    workers_per_gpu= 4,
    train=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        img_dir='images/training',
        ann_dir='annotations/training',
        pipeline=cfg.train_pipeline ),
    val=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        img_dir='images/validation',
        ann_dir='annotations/validation',
        pipeline=cfg.test_pipeline),
    test=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        img_dir='images/validation',
        ann_dir='annotations/validation',
        pipeline=cfg.test_pipeline))

# Model and data details are modified for ade dataset from coco config

# Set up working dir to save files and logs.
cfg.work_dir = 'work_dirs/test1/'

# load from checkpoint file with pretrained parameters
cfg.load_from = checkpoint_file

cfg.optimizer.lr = 0.001
cfg.runner.max_iters = 400
cfg.evaluation.interval = 200
cfg.checkpoint_config.interval = 400

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


In [23]:
# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='SyncBN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained='open-mmlab://resnet50_v1c',
    backbone=dict(
        type='ResNetV1c',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        dilations=(1, 1, 2, 4),
        strides=(1, 2, 1, 1),
        norm_cfg=dict(type='BN', requires_grad=False),
        norm_eval=False,
        style='pytorch',
        contract_dilation=True),
    decode_head=dict(
        type='ASPPHead',
        in_channels=2048,
        in_index=3,
        channels=512,
        dilations=(1, 12, 24, 36),
        dropout_ratio=0.1,
        num_classes=150,
        norm_cfg=dict(type='BN', requires_grad=False),
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
    auxiliary_head=dict(
        type='FCNHead',
        in_channels=1024,
        in_index=2,
        channels=256,
        num_convs=1,
        conc

### Train and Evaluation

In [24]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
import mmcv 
import os.path as osp


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

2021-10-23 13:49:22,228 - mmseg - INFO - Loaded 20210 images
2021-10-23 13:49:23,392 - mmseg - INFO - Loaded 2000 images
2021-10-23 13:49:23,393 - mmseg - INFO - load checkpoint from checkpoints/deeplabv3_r50-d8_512x512_4x4_20k_coco-stuff10k_20210821_043025-b35f789d.pth
2021-10-23 13:49:23,394 - mmseg - INFO - Use load_from_local loader
2021-10-23 13:49:23,590 - mmseg - WARNING - The model and loaded state dict do not match exactly

size mismatch for decode_head.conv_seg.weight: copying a param with shape torch.Size([171, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([150, 512, 1, 1]).
size mismatch for decode_head.conv_seg.bias: copying a param with shape torch.Size([171]) from checkpoint, the shape in current model is torch.Size([150]).
size mismatch for auxiliary_head.conv_seg.weight: copying a param with shape torch.Size([171, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([150, 256, 1, 1]).
size mismatch for auxiliary_head.conv_seg.bi

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 2000/2000, 1.9 task/s, elapsed: 1046s, ETA:     0s

2021-10-23 14:24:33,424 - mmseg - INFO - per class results:
2021-10-23 14:24:33,433 - mmseg - INFO - 
+---------------------+------+------+
|        Class        | IoU  | Acc  |
+---------------------+------+------+
|         wall        | 2.31 | 4.31 |
|       building      | 0.57 | 2.25 |
|         sky         | 1.12 | 3.72 |
|        floor        | 0.01 | 0.02 |
|         tree        | 0.0  | 0.0  |
|       ceiling       | 0.13 | 0.27 |
|         road        | 0.0  | 0.0  |
|         bed         | 0.0  | 0.0  |
|      windowpane     | 0.0  | 0.0  |
|        grass        | 0.0  | 0.0  |
|       cabinet       | 0.0  | 0.0  |
|       sidewalk      | 0.0  | 0.0  |
|        person       | 0.0  | 0.0  |
|        earth        | 0.0  | 0.0  |
|         door        | 0.0  | 0.0  |
|        table        | 0.0  | 0.0  |
|       mountain      | 0.0  | 0.0  |
|        plant        | 0.0  | 0.0  |
|       curtain       | 0.0  | 0.0  |
|        chair        | 0.0  | 0.0  |
|         car         | 

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 2000/2000, 1.9 task/s, elapsed: 1041s, ETA:     0s

2021-10-23 14:59:44,146 - mmseg - INFO - per class results:
2021-10-23 14:59:44,154 - mmseg - INFO - 
+---------------------+------+------+
|        Class        | IoU  | Acc  |
+---------------------+------+------+
|         wall        | 2.17 | 3.47 |
|       building      | 0.64 | 2.42 |
|         sky         | 0.59 | 1.71 |
|        floor        | 0.07 | 0.17 |
|         tree        | 0.0  | 0.0  |
|       ceiling       | 0.22 | 0.64 |
|         road        | 0.0  | 0.0  |
|         bed         | 0.0  | 0.0  |
|      windowpane     | 0.0  | 0.0  |
|        grass        | 0.01 | 0.02 |
|       cabinet       | 0.0  | 0.0  |
|       sidewalk      | 0.0  | 0.0  |
|        person       | 0.0  | 0.0  |
|        earth        | 0.01 | 0.02 |
|         door        | 0.0  | 0.0  |
|        table        | 0.0  | 0.0  |
|       mountain      | 0.0  | 0.0  |
|        plant        | 0.02 | 0.02 |
|       curtain       | 0.0  | 0.0  |
|        chair        | 0.0  | 0.0  |
|         car         | 

In [30]:
# Modifying the config file for our dataset:
from mmseg.apis import set_random_seed


# Since we use ony one GPU, BN is used instead of SyncBN
cfg.norm_cfg_1 = dict(type='BN', requires_grad=True)
cfg.norm_cfg_2 = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg_1        # Unfreezing layers
cfg.model.decode_head.norm_cfg = cfg.norm_cfg_1 
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg_2

# modify num classes of the model in decode/auxiliary head - classes - 150 in ade
cfg.model.decode_head.num_classes = 150 
cfg.model.auxiliary_head.num_classes = 150 

# Modify dataset type and path
cfg.dataset_type = 'ADE20KDataset'
cfg.data_root = 'data/ade/ADEChallengeData2016'

# cfg.data.samples_per_gpu = 4
# cfg.data.workers_per_gpu= 4

cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (512, 512)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(2048, 1024), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(2048, 512),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


cfg.data = dict(
    samples_per_gpu= 4,
    workers_per_gpu= 4,
    train=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        img_dir='images/training',
        ann_dir='annotations/training',
        pipeline=cfg.train_pipeline ),
    val=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        img_dir='images/validation',
        ann_dir='annotations/validation',
        pipeline=cfg.test_pipeline),
    test=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        img_dir='images/validation',
        ann_dir='annotations/validation',
        pipeline=cfg.test_pipeline))

# Model and data details are modified for ade dataset from coco config

# Set up working dir to save files and logs.
cfg.work_dir = 'work_dirs/test1/'

# load from checkpoint file with pretrained parameters
cfg.load_from = 'work_dirs/test1/iter_400.pth'

cfg.optimizer.lr = 0.0001 #very small learning rate
cfg.runner.max_iters = 400
cfg.evaluation.interval = 200
cfg.checkpoint_config.interval = 400

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


In [31]:
# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='SyncBN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained='open-mmlab://resnet50_v1c',
    backbone=dict(
        type='ResNetV1c',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        dilations=(1, 1, 2, 4),
        strides=(1, 2, 1, 1),
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        contract_dilation=True),
    decode_head=dict(
        type='ASPPHead',
        in_channels=2048,
        in_index=3,
        channels=512,
        dilations=(1, 12, 24, 36),
        dropout_ratio=0.1,
        num_classes=150,
        norm_cfg=dict(type='BN', requires_grad=True),
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
    auxiliary_head=dict(
        type='FCNHead',
        in_channels=1024,
        in_index=2,
        channels=256,
        num_convs=1,
        concat

In [32]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
import mmcv 
import os.path as osp


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

2021-10-23 15:28:53,655 - mmseg - INFO - Loaded 20210 images
/home/lakswa/Project/mmsegmentation/mmseg/models/backbones/resnet.py:431: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '
2021-10-23 15:28:54,819 - mmseg - INFO - Loaded 2000 images
2021-10-23 15:28:54,820 - mmseg - INFO - load checkpoint from work_dirs/test1/iter_400.pth
2021-10-23 15:28:54,821 - mmseg - INFO - Use load_from_local loader
2021-10-23 15:28:55,127 - mmseg - INFO - Start running, host: lakswa@dml-instance, work_dir: /home/lakswa/Project/mmsegmentation/work_dirs/test1
2021-10-23 15:28:55,128 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epo

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 2000/2000, 1.9 task/s, elapsed: 1042s, ETA:     0s

2021-10-23 16:04:03,783 - mmseg - INFO - per class results:
2021-10-23 16:04:03,791 - mmseg - INFO - 
+---------------------+------+------+
|        Class        | IoU  | Acc  |
+---------------------+------+------+
|         wall        | 2.02 | 3.56 |
|       building      | 0.72 | 2.72 |
|         sky         | 0.81 | 2.35 |
|        floor        | 0.01 | 0.02 |
|         tree        | 0.01 | 0.02 |
|       ceiling       | 0.11 | 0.23 |
|         road        | 0.0  | 0.0  |
|         bed         | 0.0  | 0.0  |
|      windowpane     | 0.0  | 0.0  |
|        grass        | 0.02 | 0.02 |
|       cabinet       | 0.0  | 0.0  |
|       sidewalk      | 0.0  | 0.0  |
|        person       | 0.0  | 0.0  |
|        earth        | 0.01 | 0.01 |
|         door        | 0.0  | 0.0  |
|        table        | 0.0  | 0.0  |
|       mountain      | 0.0  | 0.0  |
|        plant        | 0.04 | 0.04 |
|       curtain       | 0.0  | 0.0  |
|        chair        | 0.0  | 0.0  |
|         car         | 

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 2000/2000, 1.9 task/s, elapsed: 1041s, ETA:     0s

2021-10-23 16:39:26,151 - mmseg - INFO - per class results:
2021-10-23 16:39:26,159 - mmseg - INFO - 
+---------------------+------+------+
|        Class        | IoU  | Acc  |
+---------------------+------+------+
|         wall        | 2.6  | 4.3  |
|       building      | 0.68 | 2.39 |
|         sky         | 0.5  | 1.38 |
|        floor        | 0.08 | 0.19 |
|         tree        | 0.0  | 0.0  |
|       ceiling       | 0.19 | 0.55 |
|         road        | 0.0  | 0.0  |
|         bed         | 0.0  | 0.0  |
|      windowpane     | 0.0  | 0.0  |
|        grass        | 0.04 | 0.06 |
|       cabinet       | 0.0  | 0.0  |
|       sidewalk      | 0.0  | 0.0  |
|        person       | 0.0  | 0.0  |
|        earth        | 0.03 | 0.04 |
|         door        | 0.0  | 0.0  |
|        table        | 0.0  | 0.0  |
|       mountain      | 0.0  | 0.0  |
|        plant        | 0.05 | 0.05 |
|       curtain       | 0.0  | 0.0  |
|        chair        | 0.0  | 0.0  |
|         car         | 

In [33]:
import matplotlib.pyplot as plt

img = mmcv.imread('data/ade/ADEChallengeData2016/images/validation/ADE_val_00000907.jpg')

model.cfg = cfg
result = inference_segmentor(model, img)
plt.figure(figsize=(8, 6))
show_result_pyplot(model, img, result, get_palette('ade'))

TypeError: 'module' object is not callable